In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import scipy.stats

In [2]:
def get_data(filename):
    Completed_Job_Data = pd.read_csv(filename)

    JobAngles = Completed_Job_Data.iloc[:, 1]
    BobFidelity = Completed_Job_Data.iloc[:, 11]
    EveFidelity = Completed_Job_Data.iloc[:, 12]

    JobAngles = np.array(JobAngles)
    BobFidelity = np.array(BobFidelity)
    EveFidelity = np.array(EveFidelity)
    
    return  JobAngles, BobFidelity, EveFidelity

In [3]:
def fidelity_curves_intersection(JobAngles, BobFidelity, EveFidelity, degree):
    
    Model_Bob = np.poly1d(np.polyfit(JobAngles, BobFidelity, degree))
    Model_Eve = np.poly1d(np.polyfit(JobAngles, EveFidelity, degree))

    
    ModelDiff = Model_Bob - Model_Eve
    Roots = np.roots(ModelDiff)
    RealRoots = Roots[~np.iscomplex(Roots)]
    for j in range(len(RealRoots)):
        if RealRoots[j] >= 0 and RealRoots[j] <= np.pi/4:
            critical_theta = RealRoots[j]
            #critical_error =  error_rate_theory(critical_theta) 
            critical_error = 1 - Model_Bob(critical_theta)
            
    return critical_theta, critical_error

In [4]:
def bootstrapped_intersection_data(JobAngles, BobFid, EveFid, N):
    Index = []
    for j in range(100):
        Index.append(j)
    Index = np.array(Index)
    theta_vals = np.zeros(N)
    error_vals = np.zeros(N)
    Boot_Angles = np.zeros(100)
    Boot_BobFid = np.zeros(100)
    Boot_EveFid = np.zeros(100)
    
    for k in range(N):
        Boot_Indices = np.random.choice(Index, 100) #select the random indices to sample from
        for j in range(100):
            Boot_Angles[j] = JobAngles[Boot_Indices[j]]
            Boot_BobFid[j] = BobFid[Boot_Indices[j]]
            Boot_EveFid[j] = EveFid[Boot_Indices[j]]
            
        theta_vals[k], error_vals[k] = fidelity_curves_intersection(Boot_Angles, Boot_BobFid, Boot_EveFid, 2)
        
    return theta_vals, error_vals

In [5]:
def mean_confidence_interval(data, confidence):
    N = len(data)
    mean = np.mean(data)
    std_error = scipy.stats.sem(data)
    h = std_error * scipy.stats.t.ppf((1 + confidence)/2., N-1)
    return mean, mean - h, mean + h

In [6]:
filename = 'Experiment_July14_2023_QPU_FullSize_Ybasis_bit0.csv'

In [7]:
JobAngles, BobFid, EveFid = get_data(filename)

In [8]:
theta_vals, error_vals = bootstrapped_intersection_data(JobAngles, BobFid, EveFid, 10000)

In [9]:
angle_confidence_interval = mean_confidence_interval(theta_vals, 0.95)
error_confidence_interval = mean_confidence_interval(error_vals, 0.95)

In [10]:
print(angle_confidence_interval)

(0.4400378011267061, 0.43986313102205254, 0.44021247123135965)


In [11]:
print(error_confidence_interval)

(0.1843010096548715, 0.18422310827074992, 0.1843789110389931)
